# Files

In [4]:
from nose.tools import assert_true, assert_false, \
    assert_almost_equal, assert_equal, assert_raises

In [5]:
import os
import pickle
import gzip

**Problem 1. (20 Points):** Write a function `write_pickle` that saves an object (e.g. our dictionary of lab values) to a compressed pickle file (i.e. use gzip with pickle). Your function should take as a **positional argument** the object to be pickled and as **keyword arguments** the file to write to and a flag `write_over` that if `True` will write over an existing file and if `False` will raise a `FileExistsError`` if a file with that name exists. Test that you can both write and read again a test object (e.g. the parsed lab values from problem 1.

In [6]:
import pickle, gzip


def write_pickle(data, fname="temp.pickle", write_over=False):

    if os.path.exists(fname) and not write_over:
        raise FileExistsError("%s exists but write_over is set to False")
    with gzip.open(fname, "wb") as f0:
        pickle.dump(data, f0)  


In [7]:
if os.path.exists("temp.pickle"):
    os.remove("temp.pickle")
write_pickle("This is a test")
assert_true(os.path.exists("temp.pickle"))
assert_raises(FileExistsError, write_pickle, "This is a test", 
              write_over=False)
assert_equal(write_pickle("This is a test", write_over=True), None)
os.remove("temp.pickle")

## Test for Problem 1

In [8]:
fname = "test.pickle"
if os.path.exists(fname):
    os.remove(fname)
write_pickle("This is a test", fname=fname)
def verify(fname):
    with gzip.open(fname, "rb") as f0:
        return pickle.load(f0)
assert_equal(verify("test.pickle"), "This is a test")
os.remove("test.pickle")

**PROBLEM 2 (20 points)**:
FASTQ is a common file format in bioinformatics. Here is the description of FASTAQ from [Wikipedia](http://en.wikipedia.org/wiki/FASTQ_format)

A FASTQ file normally uses four lines per sequence.
* Line 1 begins with a '@' character and is followed by a sequence identifier and an optional description (like a FASTA title line).
* Line 2 is the raw sequence letters.
* Line 3 begins with a '+' character and is optionally followed by the same sequence identifier (and any description) again.
* Line 4 encodes the quality values for the sequence in Line 2, and must contain the same number of symbols as letters in the sequence.
A FASTQ file containing a single sequence might look like this:

Write a function `read_fastq` for reading and parsing a FASTQ file.
Your class should

1. Return a dictionary keyed on the sequence identifier with values a 3-tuple of the sequence string, comment/descriptor string, and quality string. 
    1. Use `strip` to remove any leading or trailing white spaces.
1. Uses appropriate error checking to make sure the sequence is valid. For example, our only valid characters used in the sequence? Is the quality string the same length as the sequence string? Does the the third line start with a `"+"`?

#### Hints

1. Treat the file object as an iterator and use the [`next`](https://docs.python.org/3/library/functions.html#next) function.
1. Use an infinite while loop. Break out of the loop when you get a `StopIteration` exception.

In [11]:
def read_fastq(fname):
    """
    Reads the content of a fastq file and returns the content in a dictionary.
    
    Arguments:
        fname: A string containg a path to the fastq file to read
    Returns:
        A dictionary keyed by sequence identifer with values a 
        3-tuple of strings
    
    Raises:
        ValueError if sequence string and quality string are of unequal length
        ValueError if comment string is invalid
    
    """
    
    valid = set("ACTGN")
    with open(fname) as f0:
        seqs = {}
        while True:
            try:
                line = next(f0)
                if line[0] == '@':
                    s2 = next(f0).strip()
                    if not set(s2).issubset(valid):
                        raise ValueError("Invalid sequence")
                    s3 = next(f0).strip()
                    if s3[0] != '+':
                        raise ValueError("Invalid third line of sequence")
                    s4 = next(f0).strip()
                    if len(s4) != len(s2):
                        raise ValueError(
                            "Sequence and Quality unequal lengths")
                    seqs[line.strip()] = (s2, 
                                          s3, 
                                          s4)
            except StopIteration:
                break
        return seqs

## Tests for Problem 2

In [12]:
seqs = read_fastq("ex_test.fastq")

In [13]:
assert_equal(len(seqs), 9)

In [14]:
assert_true('@A80HNBABXX:4:1:1712:2224#0/1' in seqs)

In [15]:
seqs["@A80HNBABXX:4:1:1697:2246#0/1"][2]

'^ffbffffffYddc^f`ff\\`caccddddd\\\\cccffffedcdda_c`ccb]RZ_YYY]_^`aab^^\\ccecc^d^b``L[TYQ[^_Z__^Y```cac``'

In [16]:
assert_equal(seqs["@A80HNBABXX:4:1:1697:2246#0/1"][2], 
'^ffbffffffYddc^f`ff\\`caccddddd\\\\cccffffedcdda_c`ccb]RZ_YYY]_^`aab^^\\ccecc^d^b``L[TYQ[^_Z__^Y```cac``'
)